In [1]:
from pathlib import Path
import numpy as np
import SimpleITK as sitk
from MeshMetrics.utils import (
    create_synthetic_examples_2d,
    create_synthetic_examples_3d,
    vtk_write_polydata,
    sitk2np,
    vtk_read_polydata,
    calculate_metrics_and_print_scores
)
from MeshMetrics import DistanceMetrics
# set data directory
data_dir = Path('data')

Read or create you own synthetic examples (a few are already provided in the examples folder)

In [2]:
create_data = False

# you can change the NSD and BIoU tau parameter here
tau = 20
if create_data:
    # you can change the radii of the circles/spheres here
    r1, r2 = 25, 40
    spacing2d = (0.2, 0.2)
    spacing3d = (1.0, 1.0, 1.0)
    
    # 2D: Create synthetic circles with predefined radii
    ref2d_vtk, pred2d_vtk, ref2d_sitk, pred2d_sitk = create_synthetic_examples_2d(r1=r1, r2=r2, spacing=spacing2d)
    data_dir.mkdir(exist_ok=True)
    vtk_write_polydata(ref2d_vtk, data_dir / "example_2d_ref_mesh.obj")
    vtk_write_polydata(pred2d_vtk, data_dir / "example_2d_pred_mesh.obj")
    sitk.WriteImage(ref2d_sitk, data_dir / "example_2d_ref_mask.nii.gz")
    sitk.WriteImage(pred2d_sitk, data_dir / "example_2d_pred_mask.nii.gz")
    
    # 3D: Create synthetic spheres with predefined radii
    ref3d_vtk, pred3d_vtk, ref3d_sitk, pred3d_sitk = create_synthetic_examples_3d(r1=r1, r2=r2, spacing=spacing3d)
    data_dir.mkdir(exist_ok=True)
    vtk_write_polydata(ref3d_vtk, data_dir / "example_3d_ref_mesh.obj")
    vtk_write_polydata(pred3d_vtk, data_dir / "example_3d_pred_mesh.obj")
    sitk.WriteImage(ref3d_sitk, data_dir / "example_3d_ref_mask.nii.gz")
    sitk.WriteImage(pred3d_sitk, data_dir / "example_3d_pred_mask.nii.gz")
else:
    # do not change the radii of the circles/spheres here
    r1, r2 = 25, 40
    spacing2d = (0.2, 0.2)
    spacing3d = (1.0, 1.0, 1.0)
    
    # load 2D data
    ref2d_sitk = sitk.ReadImage(data_dir / "example_2d_ref_mask.nii.gz")
    pred2d_sitk = sitk.ReadImage(data_dir / "example_2d_pred_mask.nii.gz")
    ref2d_vtk = vtk_read_polydata(data_dir / "example_2d_ref_mesh.obj")
    pred2d_vtk = vtk_read_polydata(data_dir / "example_2d_pred_mesh.obj")
    
    # load 3D data
    ref3d_sitk = sitk.ReadImage(data_dir / "example_3d_ref_mask.nii.gz")
    pred3d_sitk = sitk.ReadImage(data_dir / "example_3d_pred_mask.nii.gz")
    ref3d_vtk = vtk_read_polydata(data_dir / "example_3d_ref_mesh.obj")
    pred3d_vtk = vtk_read_polydata(data_dir / "example_3d_pred_mesh.obj")

In [3]:
# no need to read the code below, just read the output
dist_scores = abs(r1-r2)
nsd_perc = 100 if abs(r1-r2) <= tau else 0
nsd_more_less = "less" if nsd_perc == 100 else "more"
r_max = max(r1, r2)
r_min = min(r1, r2)

intersec_3d = 4/3*np.pi*max((r_min**3 - (r_max-tau)**3), 0)
union_3d = 4/3*np.pi*(r_max**3 - max(r_max-tau, r_min)**3 + r_min**3 - max(r_min-tau, 0)**3)
biou_perc_3d = intersec_3d / union_3d*100

intersec_2d = np.pi*max((r_min**2 - (r_max-tau)**2), 0)
union_2d = np.pi*(r_max**2 - max(r_max-tau, r_min)**2 + r_min**2 - max(r_min-tau, 0)**2)
biou_perc_2d = intersec_2d / union_2d*100

print(f"""
Ideally, we would observe the following:
- HD, HD_p, MASD, and ASSD should have a value of {dist_scores} mm (|r1 - r2|), 
  as all points on the two concentric spheres (in 3D) or circles (in 2D) are equidistant.
- NSD (τ = {tau} mm) should be {nsd_perc}%, as both surfaces are {nsd_more_less} than the tolerance `τ` apart.
- BIoU (τ = {tau} mm) should be {biou_perc_3d:.2f}% in 3D and {biou_perc_2d:.2f}% in 2D.

Realistically, image grids provide a discretized representation of smooth spheres or circles, 
so some deviation from the ideal values is expected. This is especially true because the grid spacing 
used is 1.0 mm, which introduces significant discretization errors on curved surfaces like those in our examples.
This is improved by using original meshes, which provide a more accurate representation of the input geometry.
""")


Ideally, we would observe the following:
- HD, HD_p, MASD, and ASSD should have a value of 15 mm (|r1 - r2|), 
  as all points on the two concentric spheres (in 3D) or circles (in 2D) are equidistant.
- NSD (τ = 20 mm) should be 100%, as both surfaces are less than the tolerance `τ` apart.
- BIoU (τ = 20 mm) should be 11.94% in 3D and 14.29% in 2D.

Realistically, image grids provide a discretized representation of smooth spheres or circles, 
so some deviation from the ideal values is expected. This is especially true because the grid spacing 
used is 1.0 mm, which introduces significant discretization errors on curved surfaces like those in our examples.
This is improved by using original meshes, which provide a more accurate representation of the input geometry.



In [4]:
# initialize the DistanceMetrics class
dist_metrics = DistanceMetrics()

# 3D examples

## Calculate metrics using `SimpleITK.Image` masks

In [5]:
# print some metadata
ref3d_sitk.GetSize(), ref3d_sitk.GetSpacing()

((90, 90, 90), (1.0, 1.0, 1.0))

In [6]:
# note that spacing is automatically inferred from the sitk.Image object
dist_metrics.set_input(ref=ref3d_sitk, pred=pred3d_sitk)
# run calculations
calculate_metrics_and_print_scores(dist_metrics, tau=tau)

HD100: 15.24 mm, HD (perc=95): 15.06 mm, MASD: 14.73 mm, ASSD: 14.74 mm, NSD (tau=20 mm): 100.00 %, BIoU (tau=20 mm): 14.34 %


## Calculate metrics using `numpy.ndarray` masks

In [7]:
# convert sitk to numpy
spacing3d = ref3d_sitk.GetSpacing()
ref3d_np = sitk2np(ref3d_sitk) > 0
pred3d_np = sitk2np(pred3d_sitk) > 0
# check masks size and spacing
ref3d_np.shape, pred3d_np.shape, spacing3d

((90, 90, 90), (90, 90, 90), (1.0, 1.0, 1.0))

In [8]:
dist_metrics.set_input(ref=ref3d_np, pred=pred3d_np, spacing=spacing3d) # do not forget to set spacing
# run calculations
calculate_metrics_and_print_scores(dist_metrics, tau=tau)

HD100: 15.24 mm, HD (perc=95): 15.06 mm, MASD: 14.73 mm, ASSD: 14.74 mm, NSD (tau=20 mm): 100.00 %, BIoU (tau=20 mm): 14.34 %


## Calculate metrics using `vtk` meshes

In [9]:
# set spacing to arbitrary value (this is only needed for BIoU calculation, the low the better, but it prolongs the computation)
dist_metrics.set_input(ref=ref3d_vtk, pred=pred3d_vtk, spacing=(0.1, 0.1, 0.1))
# run calculations
calculate_metrics_and_print_scores(dist_metrics, tau=tau)

HD100: 15.00 mm, HD (perc=95): 14.99 mm, MASD: 14.98 mm, ASSD: 14.98 mm, NSD (tau=20 mm): 100.00 %, BIoU (tau=20 mm): 12.25 %


## Advanced usage: calculate metrics using combination of `SimpleITK.Image` masks and `vtk` meshes
Please note that the `SimpleITK.Image` masks and `vtk` meshes must be in the same physical space (world coordinate system).

In [10]:
# Calculate metrics using a combination of input data, a `SimpleITK.Image` mask and a `vtk` mesh. 
# Spacing is automatically inferred from the sitk.Image object.
dist_metrics = DistanceMetrics()
dist_metrics.set_input(ref=ref3d_sitk, pred=pred3d_vtk)
# run calculations
calculate_metrics_and_print_scores(dist_metrics, tau=tau)

HD100: 15.49 mm, HD (perc=95): 15.29 mm, MASD: 14.89 mm, ASSD: 14.84 mm, NSD (tau=20 mm): 100.00 %, BIoU (tau=20 mm): 14.34 %


# 2D examples

## Calculate metrics using `SimpleITK.Image` masks

In [11]:
# print some metadata
ref2d_sitk.GetSize(), ref2d_sitk.GetSpacing()

((410, 410), (0.20000000298023224, 0.20000000298023224))

In [12]:
dist_metrics.set_input(ref=ref2d_sitk, pred=pred2d_sitk)
# run calculations
calculate_metrics_and_print_scores(dist_metrics, tau=tau)

HD100: 15.03 mm, HD (perc=95): 15.00 mm, MASD: 14.92 mm, ASSD: 14.92 mm, NSD (tau=20 mm): 100.00 %, BIoU (tau=20 mm): 14.92 %


## Calculate metrics using `numpy.ndarray` masks

In [13]:
# convert sitk to numpy
spacing2d = ref2d_sitk.GetSpacing()
ref2d_np = sitk2np(ref2d_sitk) > 0
pred2d_np = sitk2np(pred2d_sitk) > 0
# check masks size and spacing
ref2d_np.shape, pred2d_np.shape, spacing2d

((410, 410), (410, 410), (0.20000000298023224, 0.20000000298023224))

In [14]:
dist_metrics.set_input(ref=ref2d_np, pred=pred2d_np, spacing=spacing2d) # do not forget to set spacing
# run calculations
calculate_metrics_and_print_scores(dist_metrics, tau=tau)

HD100: 15.03 mm, HD (perc=95): 15.00 mm, MASD: 14.92 mm, ASSD: 14.92 mm, NSD (tau=20 mm): 100.00 %, BIoU (tau=20 mm): 14.92 %


## Calculate metrics using `vtk` meshes

In [15]:
# set spacing to arbitrary value (this is only needed for BIoU calculation, the low the better, but it prolongs the computation)
dist_metrics.set_input(ref=ref2d_vtk, pred=pred2d_vtk, spacing=(0.01, 0.01))
# run calculations
calculate_metrics_and_print_scores(dist_metrics, tau=tau)

HD100: 14.99 mm, HD (perc=95): 14.99 mm, MASD: 14.98 mm, ASSD: 14.98 mm, NSD (tau=20 mm): 100.00 %, BIoU (tau=20 mm): 14.39 %
